<a href="https://colab.research.google.com/github/yongsa-nut/TU_CN408_GenAI_671/blob/main/HW_4_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW 4 - RAG and Evaluation (10 Points)

## DUE: September 24th, 2024. 23:59.

## คำอธิบาย

- การบ้านนี้ เราจะมาฝึกใช้ RAG จริงๆ และก็จะมา evaluate ผลที่ได้ในหลายรูปแบบ
- การบ้านจะประกอบไปด้วย:
  - การ สร้าง Vector Database จาก documents ที่ให้มา
  - การทดสอบเรื่อง การตัดประโยคเก็บไว้ใน Database
  - การทดสอบ Embedding ต่างๆ
  - การทดสอบ Retrieval เกี่ยวกับ ค่า Top-k
  - การทดสอบ คำตอบ โดยใช้ LLM เป็นคนตรวจสอบ (LLM as a judge)
- การบ้านนี้น่าจะยาก ควรเริ่มทำตั้งแต่ต้นๆ เมื่อมีปัญหามาถามได้



## Load the data
- Data นี้มาจาก Huggingface [RAG Evaluation](https://huggingface.co/learn/cookbook/en/rag_evaluation#rag-evaluation)
- Data มีสองอัน: 1) อันแรกคือ Documents, 2) อันที่สองคือ คำถามที่จะต้องตอบ

In [ ]:
!pip install -U google-cloud-aiplatform "anthropic[vertex]"

In [ ]:
!pip install sentence_transformers datasets langchain
!pip3 install pinecone

In [ ]:
import pandas as pd

documents = pd.read_csv("hf://datasets/m-ric/huggingface_doc/huggingface_doc.csv")
training  = pd.read_parquet("hf://datasets/m-ric/huggingface_doc_qa_eval/data/train-00000-of-00001.parquet")

In [ ]:
documents.head()

- Documents มี สอง columns: text กับ source
- Text คือ Text ทั้ง บทความ. 1 row = 1 บทความ
- มีทั้งหมด 2647 บทความ

In [ ]:
training.head()

- training มีหลาย column หลักๆ ที่จะใช้ แค่ `question` กับ `answer`
- Columns อื่นๆมาจาก การสร้าง training data ซึ่ง training data สร้างจาก LLM โดยให้ LLM ตั้งคำถามจาก context ที่ให้ไป แล้วมีการตรวจสอบว่าคำถามตรงกับ context ไหม ซึ่งการตรวจสอบนั้น ใช้สามค่า standalone_score, relatedness_score, relevance_score. อ่านเพิ่มเติมได้ใน Post ที่ link ด้านบน

### สร้าง Data ของเรา
- เนื่องจาก documents เป็น data ขนาดใหญ่ ซึ่งจะทำให้เราใช้เวลา embedding และ upend to database นานมาก เราจะตัดออกให้เหลือแค่ 100 documents เท่านั้น
- ขั้นแรกเราจะเอา documents ที่มีอยู่ใน training มาก่อน
- ขั้นสองเราจะเพิ่ม documents เข้าไปจนครบ 100 documents

In [ ]:
source_index = documents.index[documents['source'].isin(training['source_doc'])].tolist()

doc_size = 100
i = 0
while len(source_index) < doc_size:
  if i not in source_index:
    source_index.append(i)
  i += 5 # grab every fifth documents not in source_index
source_index.sort()

small_documents = documents.iloc[source_index]
small_documents.head()

## Embedding Data

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

เราจะทดสอบ Embeddding สองตัว ดังต่อไปนี้
- 'all-MiniLM-L6-v2'
- 'BAAI/bge-m3' เป็นตัวคนใช้เยอะ เวลาใช้ SentenceTransformer (See [Documentation](https://huggingface.co/BAAI/bge-m3))

เนื่องด้วย documents ที่เราใช้นั้นใหญ่ เราจะใช้ cuda ในการรัน
- ไปที่ Runtime > Change runtime type > T4 GPU


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('No cuda!! - Embedding time will be very long!!!')

mini_embedding =  SentenceTransformer('all-MiniLM-L6-v2',  device=device)
bge_embedding =  SentenceTransformer('BAAI/bge-m3',  device=device)

ทดสอบ Embedding

In [ ]:
query = 'Today is a nice day.'
mini_ec = mini_embedding.encode(query)
bge_ec = bge_embedding.encode(query)

print(mini_ec[:5])
print(bge_ec[:5])

## Preparing the data

- ในขั้นตอนถัดไปเราจะมาเตรียมข้อมูลให้พร้อม ก่อนที่จะเอาไป upend ไปที่ database
- ขั้นตอนหลักคือเราจะต้องแบ่งบทความเป็นส่วนย่อๆแทนที่จะใช้ทั้งบทความไป embed
- เราจะทดสอบสองแบบ และจะใช้ `Langchain` library มาช่วย
 - แบบแรกคือ ตัดเป็นความยาวเท่าๆกัน ใช้ `CharacterTextSplitter` ([Documentation](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.CharacterTextSplitter.html))
 - แบบสองคือ ตัดแบบrecursiveตาม structure ใช้ `RecursiveCharacterTextSplitter` ([Documentation](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html))
- parameter ที่สำคัญคือ `chunk` หรือความยาวของประโยคที่จะตัด


In [ ]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

- ลองทดสอบกับบทความแรก

In [ ]:
first_doc = small_documents.text.loc[0]
first_doc

In [ ]:
chunk_size = 50 # Test different numbers
char_splitter = CharacterTextSplitter(chunk_size = chunk_size, chunk_overlap=0,
                                      separator='', #character that you would like to split on
                                      strip_whitespace=True)
char_splitter.split_text(first_doc)

- Default separators สำหรับ `RecursiveCharacterTextSplitter` นั้นมี แค่ `["\n\n", "\n", " ", ""]` เราเลยเพิ่ม `.` ลงไปด้วย

In [ ]:
chunk_size = 50 # Test different numbers
recur_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size,
                                                chunk_overlap=20,
                                                strip_whitespace=True,
                                                separators=["\n\n", "\n", ".", " ", ""]
                                                )
recur_splitter.split_text(first_doc)

- สามารถทดสอบดูได้ที่ web นี้ https://chunkviz.up.railway.app/
- ขนาด `chunk_size` ประมาณ 200 - 400 กำลังดี


### Creating splitted document (0.5 points)
- ถัดไปเราจะสร้างจริง เราจะตัดบทความทั้งหมดและเก็บไว้ใน list สองแบบ แบบแรกคือ ใช้ `CharacterTextSpliter` แบบสองคือ `RecursiveTextSpliter`
- `chunk_size` เราจะตั้งไว้ที่ 250
- **หมายเหตุ**: สำหรับการบ้านนี้จะทำแบบง่ายๆ สิ่งที่ควรจะทำอีกอย่างอื่น เก็บข้อมูลไว้ด้วยว่า ประโยคที่ถูกตัดออกมาจากบทความไหน

In [ ]:
splitted_docs = {'char': [],
                 'recur':[]}

chunk_size = 250
char_splitter = ...
recur_splitter = ...



In [ ]:
len(splitted_docs['char']) # 4701

In [ ]:
len(splitted_docs['recur']) # 7115

## Creating Embedded Documents (0.5 points)

- ขั้นตอนถัดไป เราจะมาสร้าง Embedded Documents สำหรับแต่ละ splited docs (`char` and `recur`) และ embedding (`mini`, `bge`)
- เพราะฉะนั้นจะมีด้วยกันทั้งหมด สี่อัน
- Code น่าจะรันนาน โดยเฉพาะถ้าคุณไม่ได้ใช้ GPU

In [ ]:
embedded_docs = { 'mini-char' : ...,
                  'mini-recur': ...,
                  'bge-char'  : ...,
                  'bge-recur' : ...}

## Pinecone Database (2 points)
- ในสวนนี้เราจะสร้าง pinecone database
- ขั้นแรกคุณจะต้องไปสมัครและเอา api มาใส่ให้เรียบร้อย
- pinecone webiste: https://www.pinecone.io/

In [ ]:
from pinecone import Pinecone, ServerlessSpec

In [ ]:
# Get secret key
from google.colab import userdata

- สร้าง 4 index สำหรับ 4 embedded_docs ที่เราสร้างไว้ และเก็บไว้ใน dict `indexes`

In [ ]:
embeds = {'mini': mini_embedding, 'bge': bge_embedding}

In [ ]:
pinecone = Pinecone(api_key=userdata.get('pinecone_key'))

INDEX_NAME = 'hw4-rag'
# Store the index in the dict
indexes = {}

...

In [ ]:
# @title Code: สำหรับสร้างเสร็จแล้วไม่ต้องสร้างใหม่
# code นี้สำหรับเวลาคุณสร้างเสร็จแล้ว ไม่ต้องสร้างใหม่อีก
embedded_list = ['mini-char', 'mini-recur', 'bge-char','bge-recur' ]
pinecone = Pinecone(api_key=userdata.get('pinecone_key'))

INDEX_NAME = 'hw4-rag'
indexes = {}
for doc in embedded_list:
  indexes[doc] = pinecone.Index(INDEX_NAME + doc)

- หลังจากสร้างเสร็จ เอาข้อมูลไปใส่บน database ตามที่สร้างไว้
- ให้อัพโหลดที่ละ 200 batch ต่อครั้ง อาจจะใช้เวลานานหน่อย
- metadatas ที่ต้องใส่คือ `text` หรือข้อความจริง

In [ ]:
batch_size = 200

...

- ตรวจสอบ pinecone database ว่าเรียบร้อยก่อนจะไปต่อ
- code ข้างบนควรรันแค่ครั้งเดียว หลังจากนั้นคุณไม่จำเป็นต้อง upend อีก เรียกใช้ได้เลย

## Retrieval Augmented Generation
- ถัดไป เราก็จะมาทำเรื่อง retrieval กัน

### Connect to Google Vertex AI
- ขั้นแรก คือสร้าง function มาเรียกใช้งาน claude

In [ ]:
!gcloud auth application-default login

In [ ]:
!gcloud auth application-default set-quota-project gen-ai-demo-3 # replace the last one with your project ID

In [ ]:
from anthropic import AnthropicVertex

project_id = "gen-ai-demo-3" # replace this with your project ID
region = "us-east5"  # Two region for Sonnet 3.5 ["us-east5", "europe-west1"]

client = AnthropicVertex(project_id=project_id, region=region)

# A simple Q&A generate code
def generate(prompt):
  response = client.messages.create(
    model="claude-3-haiku@20240307",
    max_tokens=1000,
    messages=[ { "role": "user", "content": prompt}]
  )
  return response.content[0].text

generate("Hello test test")

### RAG function (2 points)
- ในช่อง code ด้านล่างให้สร้าง function ที่รับ คำถาม, embedding model, database index, top_k
- ขั้นแรก ไปหาว่า documents ที่ใกล้คำถามที่สุดคืออะไร
- ขั้นสอง เอา documents ที่ได้มาสร้าง prompt เพื่อตอบคำถาม
- ขั้นสาม เอา prompt ไป gen response แล้วก็ return response ออกมา

In [ ]:
def RAG_response(query, model, index, top_k=1):
  pass

# ทดสอบ
query = training['question'][0]
RAG_response(query, embeds['bge'], indexs['bge-char'])

## Evaluation (5 points)
- สิ่งที่เราต้องการทดสอบมีด้วยกันทั้งหมด 3 อย่าง แต่ละอย่างมี สองค่า
  - embedding model: `mini` หรือ `bge`
  - spliting method: `char` หรือ `recur`
  - top-k: `1` หรือ `5`
- ดังนั้นจะมีทั้งหมด 8 ค่า เราจะสร้างรูปมาวนตรวจสอบและเก็บค่าของทั้งหมดไว้ใน `DataFrame` (หรือ `dict`) ดังนั้น `DataFrame` นี้จะมี 4 columns: `embedding, spliting, top-k, score` และมี 8 rows
- เราใช้ training มาตรวจสอบ
- ถัดไปในการ eval นี้ เราจะใช้ LLM มาตรวจว่าดีขนาดไหน
- สิ่งสำคัญคือ prompt. prompt ที่จะให้สร้างมีข้อกำหนดดังนี้
  - เราจะตรวจสองแค่สิ่งเดียวคือ ความถูกต้อง
  - คะแนนที่ได้จาก prompt จะต้องเป็นตัวเลข 0 - 4. 0 คือน้อยสุด (ไม่ถูกต้องเลย) 4 คือมากสุด (ถูกต้องครบถ้วน)
- หลังจากตรวจครบแล้วให้หาคะแนนเฉลี่ย และเก็บค่านั้นไว้ ใน column `score` ของแต่ละอัน.
- สุดท้าย print `DataFrame` (หรือ `dict`) ออกมา แบบไหนทำได้ดีที่สุด?
- **หมายเหตุ 1**: เราใช้ Claude 3 haiku สำหรับการบ้านนี้ ผมลัพท์ที่ได้อาจจะไม่ดีเท่าไร
- **หมายเหตุ 2**: ผลลัพท์ที่ได้เราควรจะเห็น ว่า
  - `bge` ดีกว่า `mini`
  - `recur` ควรจะดีกว่า `char`
  - `top-k = 5` ควรจะดีกว่า `top-k = 1`.
  - อาจจะไม่มาก และขึ้นอยู่กับ prompt ของคุณ
  
